In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.utils.data import DataLoader
from torch.optim import AdamW
from torch.nn import BCEWithLogitsLoss

In [2]:
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, TensorDataset
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

In [3]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cpu')

In [4]:
# Define model name and number of labels
model_name = "sampathlonka/San-BERT"
num_labels = 49  # Replace with the actual number of labels in your data

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/394 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/472k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/951k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/870 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at sampathlonka/San-BERT and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
data = pd.read_excel('/content/manipulated_data.xlsx')

In [6]:
test_split = 0.2

# Initial train and test split.
train_df, test_df = train_test_split(
    data,
    test_size=test_split,
)
print(f"Number of rows in training set: {len(train_df)}")
print(f"Number of rows in test set: {len(test_df)}")

Number of rows in training set: 824
Number of rows in test set: 207


In [7]:
# Split the data into texts and labels
train_texts = train_df['Text'].tolist()
train_labels = train_df.iloc[:, 1:].values.tolist()

In [8]:
test_split = 0.5

# Initial train and test split.
val_df, test_df = train_test_split(
    test_df,
    test_size=test_split,
)
print(f"Number of rows in training set: {len(val_df)}")
print(f"Number of rows in test set: {len(test_df)}")

Number of rows in training set: 103
Number of rows in test set: 104


In [9]:
# Split the data into texts and labels
val_texts = val_df['Text'].tolist()
val_labels = val_df.iloc[:, 1:].values.tolist()

In [10]:
#train_texts
#train_labels

In [11]:
# Tokenize data
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=512)

In [12]:
# Tokenize data
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=512)

In [13]:
#train_encodings[0]
#train_encodings

In [14]:
from torch.utils.data import Dataset
'''
class CustomDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.encodings["input_ids"])

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item["labels"] = self.labels[idx]
        return item'''

'\nclass CustomDataset(Dataset):\n    def __init__(self, encodings, labels):\n        self.encodings = encodings\n        self.labels = labels\n\n    def __len__(self):\n        return len(self.encodings["input_ids"])\n\n    def __getitem__(self, idx):\n        item = {key: val[idx] for key, val in self.encodings.items()}\n        item["labels"] = self.labels[idx]\n        return item'

In [15]:
class CustomDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.encodings["input_ids"])

    def __getitem__(self, idx):
        item = {
            'input_ids': torch.tensor(self.encodings["input_ids"][idx]),
            'attention_mask': torch.tensor(self.encodings["attention_mask"][idx]),
            'labels': torch.tensor(self.labels[idx])
        }
        return item

In [16]:
# Create custom dataset
train_dataset = CustomDataset(train_encodings, train_labels)

# Create dataloader
train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=False)

In [17]:
# Create custom dataset
val_dataset = CustomDataset(val_encodings, val_labels)

# Create dataloader
val_dataloader = DataLoader(val_dataset, batch_size=4, shuffle=False)

In [18]:
#next(iter(train_dataset))
#next(iter(train_dataloader))

In [19]:
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.BCEWithLogitsLoss()

In [ ]:
num_epochs = 10

for epoch in range(num_epochs):
    # Training loop
    model.train()  # Set the model to training mode
    train_loss = 0.0
    correct_train_predictions = 0
    total_train_samples = 0

    for batch in tqdm(train_dataloader, desc=f"Epoch {epoch+1}/{num_epochs}", leave=False):
        input_ids = batch['input_ids'].to(device)  # Move input to device
        attention_mask = batch['attention_mask'].to(device)  # Move attention mask to device
        labels = batch['labels'].to(device)  # Move labels to device

        optimizer.zero_grad()  # Clear gradients
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        loss = criterion(logits, labels.float())
        loss.backward()  # Backpropagation
        optimizer.step()  # Update model parameters

        train_loss += loss.item()

        # Calculate accuracy
        _, predicted = torch.max(logits,-2)
        correct_train_predictions += (predicted == labels).sum().item()
        total_train_samples += labels.size(0)

    # Validation loop
    model.eval()  # Set the model to evaluation mode
    val_loss = 0.0
    correct_val_predictions = 0
    total_val_samples = 0

    with torch.no_grad():
        for batch in tqdm(val_dataloader, desc=f"Validation", leave=False):
            input_ids = batch['input_ids'].to(device)  # Move input to device
            attention_mask = batch['attention_mask'].to(device)  # Move attention mask to device
            labels = batch['labels'].to(device)  # Move labels to device

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            loss = criterion(logits, labels.float())

            val_loss += loss.item()

            # Calculate accuracy
            _, predicted = torch.max(logits, -2)
            correct_val_predictions += (predicted == labels).sum().item()
            total_val_samples += labels.size(0)

    # Calculate average loss and accuracy
    avg_train_loss = train_loss / len(train_dataloader)
    avg_val_loss = val_loss / len(val_dataloader)
    train_accuracy = correct_train_predictions / total_train_samples
    val_accuracy = correct_val_predictions / total_val_samples

    # Print epoch statistics
    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {avg_train_loss}, Train Accuracy: {train_accuracy}, Val Loss: {avg_val_loss}, Val Accuracy: {val_accuracy}")

Epoch 1/10, Train Loss: 0.14193339309501415, Train Accuracy: 12.222087378640778, Val Loss: 0.14179979694577363, Val Accuracy: 44.54368932038835


Epoch 2/10, Train Loss: 0.12538395444093978, Train Accuracy: 11.858009708737864, Val Loss: 0.14404674820028818, Val Accuracy: 43.077669902912625


Epoch 3/10, Train Loss: 0.12625729683244113, Train Accuracy: 12.233009708737864, Val Loss: 0.1451267978319755, Val Accuracy: 46.689320388349515


Epoch 4/10, Train Loss: 0.12656282261013985, Train Accuracy: 12.273058252427184, Val Loss: 0.1458559761253687, Val Accuracy: 45.067961165048544


Epoch 5/10, Train Loss: 0.1266318913187506, Train Accuracy: 12.29126213592233, Val Loss: 0.1460220140333359, Val Accuracy: 46.54368932038835


Epoch 6/10, Train Loss: 0.12677538170687203, Train Accuracy: 12.342233009708737, Val Loss: 0.14618497972305006, Val Accuracy: 46.300970873786405


Epoch 7/10, Train Loss: 0.12712634492267683, Train Accuracy: 12.192961165048544, Val Loss: 0.14654725360182616, Val Accuracy: 46.41747572815534


Epoch 8/10, Train Loss: 0.1266127592947298, Train Accuracy: 12.095873786407767, Val Loss: 0.14679979045803732, Val Accuracy: 46.96116504854369


Epoch 9/10, Train Loss: 0.1267185093636073, Train Accuracy: 11.970873786407767, Val Loss: 0.14670645015744063, Val Accuracy: 46.116504854368934


Epoch 10/10, Train Loss: 0.1269237478308886, Train Accuracy: 12.768203883495145, Val Loss: 0.14663108466909483, Val Accuracy: 46.75728155339806


In [20]:
num_epochs = 40

for epoch in range(num_epochs):
    # Training loop
    model.train()  # Set the model to training mode
    train_loss = 0.0
    for batch in tqdm(train_dataloader, desc=f"Epoch {epoch+1}/ {num_epochs}", leave=False):
        input_ids = batch['input_ids'].to(device)  # Move input to device
        attention_mask = batch['attention_mask'].to(device)  # Move attention mask to device
        labels = batch['labels'].to(device)  # Move labels to device

        optimizer.zero_grad()  # Clear gradients
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        loss = criterion(logits, labels.float())
        loss.backward()  # Backpropagation
        optimizer.step()  # Update model parameters

        train_loss += loss.item()

    # Validation loop
    model.eval()  # Set the model to evaluation mode
    val_loss = 0.0
    with torch.no_grad():
        for batch in tqdm(val_dataloader, desc=f"Validation", leave=False):
            input_ids = batch['input_ids'].to(device)  # Move input to device
            attention_mask = batch['attention_mask'].to(device)  # Move attention mask to device
            labels = batch['labels'].to(device)  # Move labels to device

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            loss = criterion(logits, labels.float())

            val_loss += loss.item()

    # Calculate average loss
    avg_train_loss = train_loss / len(train_dataloader)
    avg_val_loss = val_loss / len(val_dataloader)

    # Print epoch statistics
    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {avg_train_loss}, Val Loss: {avg_val_loss}")

Epoch 1/40, Train Loss: 0.13586929436881565, Val Loss: 0.12825347368533796


Epoch 2/40, Train Loss: 0.1302375619489591, Val Loss: 0.12860354122060996


Epoch 3/40, Train Loss: 0.1303198169736029, Val Loss: 0.12835541854684168


Epoch 4/40, Train Loss: 0.13041999115093242, Val Loss: 0.12904676995598352


Epoch 5/40, Train Loss: 0.1301956618729147, Val Loss: 0.1292217689064833


...

Epoch 36/40, Train Loss: 0.13064838135705412, Val Loss: 0.12946133458843598


Epoch 37/40, Train Loss: 0.13057144232166623, Val Loss: 0.12936157589921585


Epoch 38/40, Train Loss: 0.1312387699207056, Val Loss: 0.12973289736188376


Epoch 39/40, Train Loss: 0.13048423280559698, Val Loss: 0.12976194688907036


Epoch 40/40, Train Loss: 0.13080298538925578, Val Loss: 0.1289207087113307


In [21]:
test_texts =test_df['Text'].tolist()
test_labels = test_df.iloc[:, 1:].values.tolist()
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=512)
test_dataset = CustomDataset(test_encodings, val_labels)
test_dataloader = DataLoader(val_dataset, batch_size=4, shuffle=False)

In [ ]:
#next(iter(test_dataloader))

In [22]:
import numpy as np

In [23]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

# Put the model in evaluation mode
model.eval()

# Lists to store predictions and labels
all_predictions = []
all_labels = []

# Iterate through the test dataset
for batch in tqdm(test_dataloader, desc="Testing", leave=False):
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    labels = batch['labels'].to(device)

    # Disable gradient computation
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits

        # Apply sigmoid activation and round to obtain binary predictions
        predictions = torch.sigmoid(logits).cpu().numpy()
        rounded_predictions = (predictions > 0.21).astype(int)

        # Append predictions and labels to lists
        all_predictions.extend(rounded_predictions)
        all_labels.extend(labels.cpu().numpy())

# Convert lists to numpy arrays for easier manipulation
all_predictions = np.array(all_predictions)
all_labels = np.array(all_labels)

# Calculate evaluation metrics
accuracy = accuracy_score(all_labels, all_predictions)
precision = precision_score(all_labels, all_predictions, average='weighted')
recall = recall_score(all_labels, all_predictions, average='weighted')
f1 = f1_score(all_labels, all_predictions, average='weighted')
conf_matrix = confusion_matrix(all_labels.argmax(axis=1), all_predictions.argmax(axis=1))
class_report = classification_report(all_labels, all_predictions)

# Print evaluation metrics
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")
print("Classification Report:")
print(class_report)

Accuracy: 0.11650485436893204
Precision: 0.056890835898650245
Recall: 0.18902439024390244
F1 Score: 0.08745904623225337
Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         5
           1       0.00      0.00      0.00         0
           2       0.00      0.00      0.00         2
           3       0.00      0.00      0.00         1
           4       0.00      0.00      0.00         0
           5       0.00      0.00      0.00         1
           6       0.00      0.00      0.00         0
           7       0.00      0.00      0.00         1
           8       0.00      0.00      0.00         1
           9       0.00      0.00      0.00         3
          10       0.00      0.00      0.00         1
          11       0.00      0.00      0.00         1
          12       0.00      0.00      0.00         0
          13       0.00      0.00      0.00         0
          14       0.00      0.00      0.00   

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1609: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: Undefi

In [ ]:
from huggingface_hub import notebook_login
notebook_login()